In [1]:
import numpy as np
from itertools import product, chain
import os as os

In [2]:
# convert a list of sequences into the corresponding graph matrix
def to_graph(seqs):
    num_nodes = sum([len(s) for s in seqs])
    m = np.array([[0] * num_nodes] * num_nodes)
    i = 0 # indicates where the current seq starts
    for s in seqs:
        l = len(s)
        
        # implement directed edges between nodes of same seq
        for j in range(i, i + l - 1):
            m[j][j+1] = 1
        
        # implement undirected edges between nodes of s and all other nodes
        for j in range(i, i + l):
            for k in range(num_nodes):
                if (k < i or k >= i + l):
                    m[j][k] = 1
                    m[k][j] = 1
        
        i += l
    return m

In [3]:
seqs = ['CATG', 'CAGT', 'AGTT']
m = to_graph(seqs)

# undirected edges = edges betwwen nucleotides in different sequences
undir = 0
for i in range(len(m)):
    for j in range(len(m)):
        if (m[i][j] == 1 and m[j][i] == 1):
            undir += 1
undir = int(undir / 2)
print(undir) # is correct

48


In [4]:
# Now simple mixed cylces: 
# use convention: lower sequence first: X11_21 instead of X21_11

# returns a list of all possible combinations of substrings of strings in seqs,
# such that at least one substring is longer than one letter (at least one directed edge contained)
# and at least two substrings are non-empty (at least two sequences are visited)

# generate all possible substrings including ''
def substrings(s):
    yield('')
    for i in range(len(s)):
        for j in range(i + 1, len(s) + 1):
            yield s[i:j]
            
# generate the equation from tuples of subsequences like ('0', '01', '0123')
# Only applicable for max 3 sequences
def to_equ(c):
    # refrase the tuple into the needed indices like: [('00', '00'), ('10', '11'), ('20', '23')]
    temp = [(str(i) + seq[0], str(i) + seq[-1]) for i, seq in enumerate(c) if seq] 

    # chain flattens a list of lists or tuples etc.
    if (len(temp) == 2):
        return s2.format(*chain(*temp))
    else:
        return s3.format(*chain(*temp))

# convert sequences of bases into strings of numbers, to be able to distiguish equal bases in a sequence
seq_nums = [''.join(map(str, range(len(s)))) for s in seqs]

# returns a list of all possible combinations of substrings of strings in seqs,
# such that at least one substring is longer than one letter (at least one directed edge contained)
# and at least two substrings are non-empty (at least two sequences are visited)
lss = [tup for tup in product(*map(substrings,seq_nums)) 
       if (len([c for c in tup if len(c) > 1]) >= 1 and len([c for c in tup if len(c) > 0]) >= 2)]

s = ''
# templates for 3 and 2 visited sequences
s3 = 'X{1}_{2} + X{3}_{4} + X{0}_{5} < 2;\nX{1}_{4} + X{2}_{5} + X{0}_{3} < 2'
s2 = 'X{1}_{2} + X{0}_{3} < 1'
for cycle in lss:
    s += to_equ(cycle) + ";" + '\n'
print('The equations for all simple mixed cycles:')
print(s)

The equations for all simple mixed cycles:
X10_20 + X10_21 < 1;
X10_20 + X10_22 < 1;
X10_20 + X10_23 < 1;
X10_21 + X10_22 < 1;
X10_21 + X10_23 < 1;
X10_22 + X10_23 < 1;
X11_20 + X10_20 < 1;
X11_20 + X10_21 < 1;
X11_20 + X10_22 < 1;
X11_20 + X10_23 < 1;
X11_21 + X10_21 < 1;
X11_21 + X10_22 < 1;
X11_21 + X10_23 < 1;
X11_22 + X10_22 < 1;
X11_22 + X10_23 < 1;
X11_23 + X10_23 < 1;
X12_20 + X10_20 < 1;
X12_20 + X10_21 < 1;
X12_20 + X10_22 < 1;
X12_20 + X10_23 < 1;
X12_21 + X10_21 < 1;
X12_21 + X10_22 < 1;
X12_21 + X10_23 < 1;
X12_22 + X10_22 < 1;
X12_22 + X10_23 < 1;
X12_23 + X10_23 < 1;
X13_20 + X10_20 < 1;
X13_20 + X10_21 < 1;
X13_20 + X10_22 < 1;
X13_20 + X10_23 < 1;
X13_21 + X10_21 < 1;
X13_21 + X10_22 < 1;
X13_21 + X10_23 < 1;
X13_22 + X10_22 < 1;
X13_22 + X10_23 < 1;
X13_23 + X10_23 < 1;
X11_20 + X11_21 < 1;
X11_20 + X11_22 < 1;
X11_20 + X11_23 < 1;
X11_21 + X11_22 < 1;
X11_21 + X11_23 < 1;
X11_22 + X11_23 < 1;
X12_20 + X11_20 < 1;
X12_20 + X11_21 < 1;
X12_20 + X11_22 < 1;
X12_20 + X11

In [5]:
print('Number of unique simple mixed cycles:', len(set(s.split('\n')))) # makes list unique

Number of unique simple mixed cycles: 2125


In [6]:
# objective function
all_nuc = list(product(list(map(str, range(3))), list(map(str, range(4)))))
all_comb = list(product(all_nuc, repeat = 2))
diff_seq_unique = [tup for tup in all_comb if tup[0][0] < tup[1][0]]
diff_seq_unique

obj = 'max '
for tup in diff_seq_unique:
    obj += '1*X' + ''.join(tup[0]) + '_' + ''.join(tup[1]) + '+'
obj[:-1]

'max 1*X00_10+1*X00_11+1*X00_12+1*X00_13+1*X00_20+1*X00_21+1*X00_22+1*X00_23+1*X01_10+1*X01_11+1*X01_12+1*X01_13+1*X01_20+1*X01_21+1*X01_22+1*X01_23+1*X02_10+1*X02_11+1*X02_12+1*X02_13+1*X02_20+1*X02_21+1*X02_22+1*X02_23+1*X03_10+1*X03_11+1*X03_12+1*X03_13+1*X03_20+1*X03_21+1*X03_22+1*X03_23+1*X10_20+1*X10_21+1*X10_22+1*X10_23+1*X11_20+1*X11_21+1*X11_22+1*X11_23+1*X12_20+1*X12_21+1*X12_22+1*X12_23+1*X13_20+1*X13_21+1*X13_22+1*X13_23'

In [7]:
# now edit per hand which nucleotides match 
obj_func = 'max: 4*X00_10+1*X00_11+1*X00_12+1*X00_13+1*X00_20+1*X00_21+1*X00_22+1*X00_23+1*X01_10+4*X01_11+1*X01_12+1*X01_13+4*X01_20+1*X01_21+1*X01_22+1*X01_23+1*X02_10+1*X02_11+1*X02_12+4*X02_13+1*X02_20+1*X02_21+4*X02_22+4*X02_23+1*X03_10+1*X03_11+4*X03_12+1*X03_13+1*X03_20+4*X03_21+1*X03_22+1*X03_23+1*X10_20+1*X10_21+1*X10_22+1*X10_23+4*X11_20+1*X11_21+1*X11_22+1*X11_23+1*X12_20+4*X12_21+1*X12_22+1*X12_23+1*X13_20+1*X13_21+4*X13_22+4*X13_23;'
obj_func

'max: 4*X00_10+1*X00_11+1*X00_12+1*X00_13+1*X00_20+1*X00_21+1*X00_22+1*X00_23+1*X01_10+4*X01_11+1*X01_12+1*X01_13+4*X01_20+1*X01_21+1*X01_22+1*X01_23+1*X02_10+1*X02_11+1*X02_12+4*X02_13+1*X02_20+1*X02_21+4*X02_22+4*X02_23+1*X03_10+1*X03_11+4*X03_12+1*X03_13+1*X03_20+4*X03_21+1*X03_22+1*X03_23+1*X10_20+1*X10_21+1*X10_22+1*X10_23+4*X11_20+1*X11_21+1*X11_22+1*X11_23+1*X12_20+4*X12_21+1*X12_22+1*X12_23+1*X13_20+1*X13_21+4*X13_22+4*X13_23;'

In [8]:
o = "int "

for tup in diff_seq_unique:
    o += 'X' + ''.join(tup[0]) + '_' + ''.join(tup[1]) + ', '
o[:-1]+';'

'int X00_10, X00_11, X00_12, X00_13, X00_20, X00_21, X00_22, X00_23, X01_10, X01_11, X01_12, X01_13, X01_20, X01_21, X01_22, X01_23, X02_10, X02_11, X02_12, X02_13, X02_20, X02_21, X02_22, X02_23, X03_10, X03_11, X03_12, X03_13, X03_20, X03_21, X03_22, X03_23, X10_20, X10_21, X10_22, X10_23, X11_20, X11_21, X11_22, X11_23, X12_20, X12_21, X12_22, X12_23, X13_20, X13_21, X13_22, X13_23,;'

In [9]:
declaration = 'int X00_10, X00_11, X00_12, X00_13, X00_20, X00_21, X00_22, X00_23, X01_10, X01_11, X01_12, X01_13, X01_20, X01_21, X01_22, X01_23, X02_10, X02_11, X02_12, X02_13, X02_20, X02_21, X02_22, X02_23, X03_10, X03_11, X03_12, X03_13, X03_20, X03_21, X03_22, X03_23, X10_20, X10_21, X10_22, X10_23, X11_20, X11_21, X11_22, X11_23, X12_20, X12_21, X12_22, X12_23, X13_20, X13_21, X13_22, X13_23;'

In [10]:
a = ""

for tup in diff_seq_unique:
    a += 'X' + ''.join(tup[0]) + '_' + ''.join(tup[1]) + ' < 1;'
a[:-1]

'X00_10 < 1;X00_11 < 1;X00_12 < 1;X00_13 < 1;X00_20 < 1;X00_21 < 1;X00_22 < 1;X00_23 < 1;X01_10 < 1;X01_11 < 1;X01_12 < 1;X01_13 < 1;X01_20 < 1;X01_21 < 1;X01_22 < 1;X01_23 < 1;X02_10 < 1;X02_11 < 1;X02_12 < 1;X02_13 < 1;X02_20 < 1;X02_21 < 1;X02_22 < 1;X02_23 < 1;X03_10 < 1;X03_11 < 1;X03_12 < 1;X03_13 < 1;X03_20 < 1;X03_21 < 1;X03_22 < 1;X03_23 < 1;X10_20 < 1;X10_21 < 1;X10_22 < 1;X10_23 < 1;X11_20 < 1;X11_21 < 1;X11_22 < 1;X11_23 < 1;X12_20 < 1;X12_21 < 1;X12_22 < 1;X12_23 < 1;X13_20 < 1;X13_21 < 1;X13_22 < 1;X13_23 < 1'

In [11]:
binary = ""
char = ";"

for x in a:
    if x == char:
        binary += x + '\n'
    else: binary += x      
print(binary)

X00_10 < 1;
X00_11 < 1;
X00_12 < 1;
X00_13 < 1;
X00_20 < 1;
X00_21 < 1;
X00_22 < 1;
X00_23 < 1;
X01_10 < 1;
X01_11 < 1;
X01_12 < 1;
X01_13 < 1;
X01_20 < 1;
X01_21 < 1;
X01_22 < 1;
X01_23 < 1;
X02_10 < 1;
X02_11 < 1;
X02_12 < 1;
X02_13 < 1;
X02_20 < 1;
X02_21 < 1;
X02_22 < 1;
X02_23 < 1;
X03_10 < 1;
X03_11 < 1;
X03_12 < 1;
X03_13 < 1;
X03_20 < 1;
X03_21 < 1;
X03_22 < 1;
X03_23 < 1;
X10_20 < 1;
X10_21 < 1;
X10_22 < 1;
X10_23 < 1;
X11_20 < 1;
X11_21 < 1;
X11_22 < 1;
X11_23 < 1;
X12_20 < 1;
X12_21 < 1;
X12_22 < 1;
X12_23 < 1;
X13_20 < 1;
X13_21 < 1;
X13_22 < 1;
X13_23 < 1;



In [12]:
f = open("MSA.lp", "a")
for x in obj_func:
    f.write(x)

for x in s:
    f.write(x)

for x in binary:
    f.write(x)
    
for x in declaration:
    f.write(x)

f.close()